## Code for solving the game Lights Out, in O(n^2)

In [32]:
import numpy as np
import copy
import math
import time
import itertools
import random

def create_puzzle(rows, cols):
    matrix = np.arange(rows * cols)
    matrix = matrix.reshape(rows, cols)
    matrix[:] = 0
    return matrix

def print_2d_matrix(matrix):
    print(matrix)

def perform_move(matrix, length, width, x, y):
    '''
    inputs:
        the x,y coordinate to invert
    perform_move - invert current tile, and the left, right, up, down tiles

    returns:
        nothing
    '''
   
    matrix[x][y] = not matrix[x][y]  # double check this
    if (x != 0):
        matrix[x - 1][y] = not matrix[x - 1][y]
    if (x != (width - 1)):
        matrix[x + 1][y] = not matrix[x + 1][y]
    if (y != 0):
        matrix[x][y - 1] = not matrix[x][y - 1]
    if (y != (length - 1)):
        matrix[x][y + 1] = not matrix[x][y + 1]
        
def is_solved(matrix):
    '''
    checks if the game is solved
    returns:
        False if an entry is 1
    '''
    is_solved = True
    for col in matrix:
        for row in col:
            if (row == 1):
                return False
    return is_solved

def next_cell(x, y, width):
    if (x == width - 1):
        x = 0
        if (y == width - 1):
            return None
        else:
            y = y + 1
    else:
        x = x + 1
    return [x,y]

In [33]:
def get_upper_triangular(A,n,m):
    '''
    input:
    A - n by n matrix
    output:
    matrix A in upper triangular form
    Modified from regular gaussian elimination algorithm
    '''
    if n != m:
        return
    
    # pivot column
    pivot_col = 0
    
    # Iterate through rows until the pivot col entry is 1
    for pivot_col in range(0,m): # m+1?
        max_i = pivot_col
        
        for row in range(pivot_col,n):

            if A[row,pivot_col] == 1: 
                max_i = row
                break

        if A[max_i,pivot_col] == 1:
            # Swap rows i and max_i
            A[[pivot_col, max_i]] = A[[max_i, pivot_col]]
            
            for u in range(pivot_col+1,m):
                if A[u][pivot_col] == 1:
                    A[u] = (A[pivot_col]+A[u])%2
        #else:
            # more than one solution
            # print("More than one solution for var")
            # print(A)
            
    return A

def back_sub(A,n,m):
    '''
    A an n by m+1 matrix, in upper triangular form
    '''
    
    x = np.zeros((n,), dtype=int)
    for i in range(n-1,-1,-1):
        if A[i,i] == 1:
            x[i] = A[i,n]
            for j in range(i+1,n):
                x[i] = (x[i] - x[j]*A[i][j]) % 2
        #else:
        #    print("error :(")
    return x
    

def is_upper_triangular(A, n, m):
    '''
    A an n by m matrix, in upper triangular form
    '''
    #A = np.array([[1, 0, 0, 1], [0, 1, 1, 0], [0, 0, 1, 1]])
    #n = 3
    #m = 3
    min = 0
    for row in A[1:]:
        for i in range(0,n):
            if row[i] == 1 and i > min:
                min = i
                break
            elif i <= min and row[i] == 1:
                return False
            elif row[i] != 0:
                return False
    return True

In [34]:
def get_matrix_from_game(n,m):
    '''
    Create n^2 x n^2 matrix for solving the game
    '''
    
    mat = np.zeros((n*n,n*n),dtype=int)

    for i in range(0,n*n):
        x = math.floor(i/n)
        y = (i)%n

        # Light up square

        mat[i][i] = 1

        if i%n != 0:
            # not left edge
            mat[i-1][i] = 1

        if i%n != n-1:
            # not right edge
            mat[i+1][i] = 1
                
        if i >= n:
            # Not top edge
            mat[i-n][i] = 1
                
        if i < n*(n-1):
            # Not bottom edge
            mat[i+n][i] = 1
    return mat

## We can test whether the upper triangular matrix is correct, by comparing it with the original

In [36]:
def get_upper_triangular_test():
    n = 3
    m = 3
    A = get_matrix_from_game(n,m)
    matrix = np.array([[0, 0, 0], [1, 1, 0], [1, 0, 1]])
    b = np.reshape(matrix, (n*m,1))
    C = np.concatenate((A,b),1)
    res = get_upper_triangular(C,n*m,n*m)
    return res

result = get_upper_triangular_test()
n = 3
m = 3
A = get_matrix_from_game(n,m)

## Test get_matrix_for_game() with a 2x2 and a 3x3 game matrix

In [37]:
class Game:
    def create_game_matrix(self):
        return create_puzzle()

    def __init__(self, length, width):
        self.length = length
        self.width = width
        self.matrix = create_puzzle(length, width)
        self.moves = create_puzzle(length, width)
        self.solver = Game_solver(length, width) # new

    def init_move_matrix(self):
        '''
        init_move_matrix - create a zeros matrix to store moves
        '''
        for i in range(0, self.width):
            for j in range(0, self.length):
                rand_value = np.random.choice([0, 1])
                self.moves[i][j] = rand_value

    def scramble(self):
        '''
        scramble - randomly generate a 1 or 0 value for each tile in the grid
        add the value to the corresponding tiles.
        '''
        self.init_move_matrix()
        for x in range(0,self.length):
            for y in range(0,self.width):
                if (self.moves[x][y] == 1):
                     perform_move(self.matrix,self.length, self.width, x, y)
        self.solver.set_game_vector(self.matrix)

    def scramble_debug(self):
        self.matrix = np.array([[0, 0, 0], [1, 1, 0], [1, 0, 1]])
        self.solver.set_game_vector(self.matrix)
        self.init_move_matrix()
        
    def next_cell(self,row,col):
        next_row = row + 1
        next_col = col + 1
        if (row == self.width - 2):
            next_row = 0
        if (col == self.length - 2):
            next_col = 0
        return [next_row, next_col]
    
    def get_hint(self):
        hint = self.solver.get_hint(self.matrix,self.length,self.width)
        return hint
    
    def solve(self):
        return self.solver.solve(self.matrix,self.length,self.width)

In [38]:
class Game_solver:
    
    def __init__(self,length,width):
        '''
        something
        '''
        self.matrix = get_matrix_from_game(length, width)
        self.game_vector = None
        self.solution = None
        self.length = length
        self.width = width

    def set_game_vector(self,game_matrix):
        '''
        Creates a vector such that each element is 1 if the corresponding square is on, otherwise 0
        '''
        self.game_vector = np.reshape(game_matrix, (self.length*self.width,1)) #.flatten('F')
    
    def solve(self,A,n,m):
        '''
        return all moves to solve the game
        '''
        A = np.concatenate((self.matrix,self.game_vector),1)
        A_tri = get_upper_triangular(A,n*m,n*m)
    
        if True: #is_upper_triangular(A_tri,n*m,n*m):
            x = back_sub(A,n*m,n*m)
        self.solution = x
        return self.solution
        
    def get_hint(self,matrix,length,width):
        '''
        return one of the moves to solve the game
        '''
        self.solve(matrix,length,width)
        print("sol",self.solution)
        indices = []
        if self.solution.any():
            for idx, item in enumerate(self.solution):
                if item == 1:
                    indices.append(idx)
        if indices:
             return random.choice(indices)
        else:
            return

In [39]:
def hint_test():
    print("Starting Lights Out!\n")
    '''
    Initialize the counters for 100 random puzzles
    '''
    length = 4
    width = length
    
    '''
    Loop through 100 random puzzles
    '''
    num_puzzles = 4
    failed_tests = 0
    for tests in range (0,num_puzzles):
        
        # Initialize a random matrix
        game = Game(length, width)
        game.scramble()
        
        hint = game.get_hint()
        print(game.matrix)
        print(hint)

    print(failed_tests, "tests failed,", num_puzzles - failed_tests, "tests passed.")
hint_test()

Starting Lights Out!

sol [0 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0]
[[1 0 0 0]
 [0 1 1 0]
 [1 0 1 1]
 [0 1 1 0]]
9
sol [1 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0]
[[1 1 0 1]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 1 1]]
10
sol [0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0]
[[0 0 0 1]
 [0 0 0 0]
 [1 0 1 1]
 [0 1 1 0]]
10
sol [0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0]
[[1 0 1 0]
 [0 1 0 0]
 [0 0 1 0]
 [1 0 1 0]]
8
0 tests failed, 4 tests passed.


In [40]:
def main():
    print("Starting Lights Out!\n")
    '''
    Initialize the counters for 100 random puzzles
    '''
    length = 5
    width = length
    
    '''
    Loop through 100 random puzzles
    '''
    num_puzzles = 10
    failed_tests = 0
    for tests in range (0,num_puzzles):
        
        # Initialize a random matrix
        game = Game(length, width)
        game.scramble()
        
        solution = game.solve()

        if (not np.array_equal(  np.reshape(game.matrix, (1, length*width)).flatten()  , (np.matmul(game.solver.matrix,solution))%2) ):
            print("fail")
            print("Game matrix",  game.matrix )
            print("Solution\n", solution)
            print("res\n",(np.matmul(game.solver.matrix,solution))%2)
            failed_tests += 1
    print(failed_tests, "tests failed,", num_puzzles - failed_tests, "tests passed.")

if __name__ == '__main__':
    main()

Starting Lights Out!

0 tests failed, 10 tests passed.


In [41]:
import PySimpleGUI as sg
# https://stackoverflow.com/questions/60352034/is-there-a-way-to-update-the-grid-in-pysimplegui-after-clicking-on-it

sg.theme('DarkAmber')
        
# Initialize a game matrix
length = 4
width = length


layout = [[sg.B(str(j*length+i), size=(8,4), key=(i,j), button_color=('black','white')) for i in range(length)] for j in range(width)]
layout2 = [sg.B('Start'), sg.B('Hint')]
layout.append(layout2)

# Create the Window
window = sg.Window('Window Title', layout)

def update_gui():
    for i, row in enumerate(game.matrix):
        for j, value in enumerate(row):
            if value == 1:
                layout[i][j].update(button_color=('yellow', 'purple'))
            else:
                layout[i][j].update(button_color=('black','white'))

def animate_moves():
    
    window.Read(timeout=1000)
    
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    if event == sg.WIN_CLOSED or event == 'Cancel':	# if user closes window or clicks cancel
        break
    
    game = Game(length, width)
    
    text = window[event].get_text()
    
    if text == 'Start':
        game.scramble()
        update_gui()
    else:
        continue
                        
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Cancel':
            break

        text = window[event].get_text()
        
        if text == 'Hint':
            hint = game.get_hint()
            print(hint)
        elif text == 'Start':
            game.scramble()
            update_gui()
        else:
            num = int(text)
            x = (num)%length
            y = math.floor(num/length)
            perform_move(game.matrix, game.length, game.width, y, x)
            update_gui()

        if is_solved(game.matrix):
            print("won")
window.close()

won
